In [1]:
%%capture
!pip install carbontracker

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split
import torch.nn.init as init
import numpy as np
import random
import matplotlib.pyplot as plt
import time
from carbontracker.tracker import CarbonTracker

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [5]:
batch_size = 128

In [6]:
transform_teacher = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize(224),
    ])

transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
transform_tinyconv = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [7]:
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_dataset_teacher = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_teacher)
test_dataset_tinyconv = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_tinyconv)

100%|██████████| 170498071/170498071 [00:03<00:00, 45886918.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [8]:
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_loader_teacher = DataLoader(test_dataset_teacher, batch_size=batch_size, shuffle=False)
test_loader_tinyconv = DataLoader(test_dataset_tinyconv, batch_size=batch_size, shuffle=False)

In [9]:
def print_stats(model, data_loader = test_loader):
  total_params = sum(p.numel() for p in model.parameters())
  print (f"Number of parameters: {total_params} \n")

  correct_test = 0
  total_test = 0
  start = time.time()
  with torch.no_grad():
    for data in data_loader:
      inputs, labels = data[0].to(device), data[1].to(device)

      outputs = model(inputs)
      # loss = criterion(outputs, labels)
      # test_current_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      total_test += labels.size(0)
      correct_test += (predicted == labels).sum().item()

  end = time.time()

  print(f"Computation time: {end - start} \n")
  print(f"Accuracy: {correct_test / total_test}")

## Teacher

In [10]:
teacher = models.resnet50()
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
teacher.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/teacher_finetuned_cifar10_v2.pth'))
teacher = teacher.to(device)

In [12]:
tracker = CarbonTracker(epochs=1)
tracker.epoch_start()
print_stats(teacher, test_loader_teacher)
tracker.epoch_end()

CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
Number of parameters: 23528522 

CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
Computation time: 45.32993674278259 

Accuracy: 0.9512
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: Average carbon intensity during training was 357.20 gCO2/kWh at detected location: The Dalles, Oregon, US.
CarbonTracker: Average carbon intensity during training was 357.20 gCO2/kWh at detected location: The Dalles, Oregon, US.
CarbonTracker: 
Actual con

## Resnet 18 Student

In [19]:
student = models.resnet18()
student.fc = nn.Linear(student.fc.in_features, 10)
student.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/resnet18_student_learning_intermediate.pth'))
student = student.to(device)

In [20]:
tracker = CarbonTracker(epochs=1)
tracker.epoch_start()
print_stats(student)
tracker.epoch_end()

CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
Number of parameters: 11181642 

CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to averag

## Tiny ConvNet

In [16]:
class studentNN(nn.Module):
    def __init__(self):
        super(studentNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, 10)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [17]:
student = studentNN().to(device)
student.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/combined_kd_inter.pth'))
student = student.to(device)

In [18]:
tracker = CarbonTracker(epochs=1)
tracker.epoch_start()
print_stats(student, test_loader_tinyconv)
tracker.epoch_end()

CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
Number of parameters: 25786186 

CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: The following components were found: GPU with device(s) Tesla T4.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.2021 gCO2/kWh.
Computation time: 29.329994678497314 

Accuracy: 0.7624
CarbonTracker: WARNING - Failed to retrieve carbon intensity: Defaulting to average carbon intensity 357.20